# The transport-length hillslope diffuser

*(Updated by Greg Tucker, CU Boulder, November 2025)*

This Jupyter notebook illustrates running the transport-length-model hillslope diffusion component in a simple example.

# The Basics

This component uses an approach similar to the Davy and Lague (2009) equation for fluvial erosion and transport, and applies it to hillslope diffusion. The formulation and implementation were inspired by Carretier et al. (2016); see this paper and references therein for justification.

The purpose of this formulation is to overcome a limitation in diffusion theory. Diffusion theory is founded on the assumption that the motions of individual particles are short relative to the length of the system of interest. In the case of diffusion theory applied to soil transport, the assumption means that the movement of a typical sediment grain during a transport "event" (such as displacement by a raindrop impact, a burrowing animal, a frost-heave episode, a tree-throw occurrence, etc.) is only a tiny fraction of the length of the hillslope. A burrowing rodent, for example, might shift sediment by a few decimeters, which we might consider to be small relative to a 100-meter long hillslope. 

Some soil motions, however, are relatively long. For example, if an animal dislodges a stone on a very steep hillslope, that same stone might cascade all the way to the base of the slope, ending up far from the location where motion was initiated. This could be called a "non-local" transport event, because the stone's trajectory does not depend only on the slope at the point of initiation but also on the characteristics of the terrain that it encounters along the way.

The purpose of this "non-local" theory, therefore, is to describe the effect of such non-local motion in a continuum mechanics framework, without needing to track individual particles.


## Theory

The elevation $z$ of a point of the landscape changes according to:

\begin{equation}
\frac{\partial z}{\partial t} = D -\epsilon + B
\end{equation}
where $\epsilon$ is the local erosion (or *entrainment*) rate [*L/T*], $D$ the local deposition rate [*L/T*], $B$ the is rate of vertical motion of material relative to a given baselevel (sometimes called "rock uplift", whether or not any tectonic processes are involved) [*L/T*]. Thus, the elevation variation results from the difference between local rates of detachment and deposition. 

The entrainment rate $\epsilon$ is taken to be a function of the slope gradient:

\begin{equation}
\epsilon = \kappa S
\end{equation}

Specifically, $S$ is the magnitude of the gradient defined as positive downhill. $\kappa$ is an entrainment rate parameter with dimensions of [*L/T*]. (For more on the interpretation of $\kappa$, see Furbish et al. (2009); for a cellular automaton implementation, see Tucker et al. (2018).)

The deposition rate depends on the sediment flux per unit width, $q_s$, at the point in question [*L$^2$/T*] multiplied by the **inverse transport length**, $\phi$ [1/*L*]:
\begin{equation}
D = \phi q_s
\end{equation}

The inverse transport length, $\phi$, represents the inverse of a characteristic transport length scale. (It is cast here as an inverse length in order to gracefully handle the case when the transport length  approaches infinity.) The inverse transport length is defined in such a way as to vary smoothly between zero (representing complete mobility with no possibility of deposition) and $1 / L_c$, where $L_c$ is a characteristic displacement length (representing displacement on a horizontal surface).


\begin{equation}
\phi = \frac{\max (1-({S}/{S_c})^2, 0)}{L_c}
\end{equation}

where $S_c$ is a critical slope [*L/L*], which would be similar to an angle of repose: when $S\ge S_c$, $\phi=0$, and any sediment flowing toward the point in question (that is, any $q_s$) passes through without any deposition. On the other hand, if the slope is horizontal ($S=0$), any sediment arriving at the point in question will be deposited over a length scale $L_c$. (Note: in the numerical implementation in this component, the deposition length scale is kept at grid spacing at minimum, to avoid numerical artifacts.)

With this formulation:
- when $S \ll S_c$, most of the sediment entering a node is deposited there, representing the pure diffusion case. In this case, the sediment flux $q_s$ does not include sediment eroded from above and is thus "local".
- when $S \approx S_c$, $\phi$ becomes zero and there is no redeposition on the node; the sediments are transferred further downstream.  This behaviour corresponds to mass wasting, whereby grains can travel a long distance before being deposited. In that case, the flux $q_s$ is "non-local" as it incorporates sediments that have both been detached locally and transited from upslope.
- for an intermediate $S$, there is a progressive transition between pure creep and "ballistic" transport of the material. This is consistent with experiments (Roering et al., 2001; Gabet and Mendoza, 2012).



## Contrast with the non-linear diffusion model

Previous models typically use a "non-linear" diffusion model proposed by different authors (e.g. Andrews and Hanks, 1985; Hanks, 1999; Roering et al., 1999) and supported by $^{10}$Be-derived erosion rates (e.g. Binnie et al., 2007) or experiments (Roering et al., 2001). It is usually presented in the following form:


\begin{equation} 
\frac{\partial z}{\partial t} = \frac{\partial q_s}{\partial x}  
\end{equation}



\begin{equation}
q_s = \frac{\kappa' S}{1-({S}/{S_c})^2}
\end{equation}

where $\kappa'$ [*L$^2$/T*] is a diffusion coefficient.

This description is thus based on the definition of a flux of transported sediment parallel to the slope:
- when the slope is small, this flux refers to diffusion-like processes such as biogenic soil disturbance, rain splash, or diffuse runoff
- when the slope gets closer to the specified critical slope, the flux increases dramatically, simulating on average the cumulative effect of mass wasting events.

Despite these conceptual differences, equations for 'non-local' soil flux predict similar topographic evolution to the 'non-linear' diffusion equations. To appreciate the similarity, consider the equilibrium case in which deposition and entrainment rates are in balance, such that $D = \epsilon$. Expanding these expressions and solving for $q_s$, we find that for the case $S < S_c$,

$$
q_s = \frac{k L_c}{1 - (S/S_c)^2} S
$$

Hence, at transport equilibrium, $\kappa' = k L_c$.

The two formulations are compared in the following example.

# Example 1:



First, we import what we'll need:

In [ ]:
import numpy as np
from matplotlib.pyplot import figure, plot, title, xlabel, ylabel

from landlab import RasterModelGrid
from landlab.components import FlowDirectorSteepest, TransportLengthHillslopeDiffuser
from landlab.plot import imshow_grid

# to plot figures in the notebook:
%matplotlib inline

Make a grid and set boundary conditions:

In [ ]:
mg = RasterModelGrid(
    (20, 20), xy_spacing=50.0
)  # raster grid with 20 rows, 20 columns and dx=50m
z = np.random.rand(mg.size("node"))  # random noise for initial topography
mg.add_field("topographic__elevation", z, at="node")

mg.set_closed_boundaries_at_grid_edges(
    False, True, False, True
)  # N and S boundaries are closed, E and W are open

Set the initial and run conditions:

In [ ]:
total_t = 2000000.0  # total run time (yr)
dt = 1000.0  # time step (yr)
nt = int(total_t // dt)  # number of time steps
uplift_rate = 0.0001  # uplift rate (m/yr)

kappa = 0.001  # erodibility (m/yr)
Sc = 0.6  # critical slope

Instantiate the components:
The hillslope diffusion component must be used together with a flow router/director that provides the steepest downstream slope for each node, with a D4 method (creates the field *topographic__steepest_slope* at nodes).

In [ ]:
fdir = FlowDirectorSteepest(mg)
tl_diff = TransportLengthHillslopeDiffuser(mg, erodibility=kappa, slope_crit=Sc)

Run the components for 2 Myr and trace an East-West cross-section of the topography every 100 kyr:

In [ ]:
for t in range(nt):
    fdir.run_one_step()
    tl_diff.run_one_step(dt)
    z[mg.core_nodes] += uplift_rate * dt  # add the uplift

    # add some output to let us see we aren't hanging:
    if t % 100 == 0:
        print(t * dt)

        # plot east-west cross-section of topography:
        x_plot = range(0, 1000, 50)
        z_plot = z[100:120]
        figure("cross-section")
        plot(x_plot, z_plot)

figure("cross-section")
title("East-West cross section")
xlabel("x (m)")
ylabel("z (m)")

And plot final topography:

In [ ]:
figure("final topography")
im = imshow_grid(
    mg, "topographic__elevation", grid_units=["m", "m"], var_name="Elevation (m)"
)

This behaviour corresponds to the evolution observed using a classical non-linear diffusion model.

# Example 2: 

In this example, we show that when the slope is steep ($S \ge S_c$), the transport-length hillsope diffusion simulates mass wasting, with long transport distances.

First, we create a grid: the western half of the grid is flat at 0 m of elevation, the eastern half is a 45-degree slope.


In [ ]:
# Create grid and topographic elevation field:
mg2 = RasterModelGrid((20, 20), xy_spacing=50.0)

z = np.zeros(mg2.number_of_nodes)
z[mg2.node_x > 500] = mg2.node_x[mg2.node_x > 500] / 10
mg2.add_field("topographic__elevation", z, at="node")

# Set boundary conditions:
mg2.set_closed_boundaries_at_grid_edges(False, True, False, True)

# Show initial topography:
im = mg2.imshow(
    "topographic__elevation", grid_units=["m", "m"], var_name="Elevation (m)"
)

# Plot an east-west cross-section of the initial topography:
z_plot = z[100:120]
x_plot = range(0, 1000, 50)
figure(2)
plot(x_plot, z_plot)
title("East-West cross section")
xlabel("x (m)")
ylabel("z (m)")

Set the run conditions:

In [ ]:
total_t = 1000000.0  # total run time (yr)
dt = 1000.0  # time step (yr)
nt = int(total_t // dt)  # number of time steps

Instantiate the components:

In [ ]:
fdir = FlowDirectorSteepest(mg2)
tl_diff = TransportLengthHillslopeDiffuser(mg2, erodibility=0.001, slope_crit=0.6)

Run for 1 Myr, plotting the cross-section regularly:

In [ ]:
for t in range(nt):
    fdir.run_one_step()
    tl_diff.run_one_step(dt)

    # add some output to let us see we aren't hanging:
    if t % 100 == 0:
        print(t * dt)
        z_plot = z[100:120]
        figure(2)
        plot(x_plot, z_plot)

The material is diffused from the top and along the slope and it accumulates at the bottom, where the topography flattens.

As a comparison, the following code uses linear diffusion on the same slope:

In [ ]:
# Import Linear diffuser:
from landlab.components import LinearDiffuser

# Create grid and topographic elevation field:
mg3 = RasterModelGrid((20, 20), xy_spacing=50.0)
z = np.ones(mg3.number_of_nodes)
z[mg.node_x > 500] = mg.node_x[mg.node_x > 500] / 10
mg3.add_field("topographic__elevation", z, at="node")

# Set boundary conditions:
mg3.set_closed_boundaries_at_grid_edges(False, True, False, True)

# Instantiate components:
fdir = FlowDirectorSteepest(mg3)
diff = LinearDiffuser(mg3, linear_diffusivity=0.1)

# Set run conditions:
total_t = 1000000.0
dt = 1000.0
nt = int(total_t // dt)

# Run for 1 Myr, plotting east-west cross-section regularly:
for t in range(nt):
    fdir.run_one_step()
    diff.run_one_step(dt)

    # add some output to let us see we aren't hanging:
    if t % 100 == 0:
        print(t * dt)
        z_plot = z[100:120]
        figure(2)
        plot(x_plot, z_plot)